In [ ]:
def split_it(text):
    splt = text.split()
    bad_chars = [';', ':'," ", "!", "*", "(", ")", '\"', ".", ",", "/", ">", "<"]

    for i in range(len(splt)):
        for b in bad_chars:
            if b in splt[i]:
                splt[i] = splt[i].replace(b, '')
        splt[i]= splt[i].lower()
    
    return splt

In [ ]:

from pathlib import Path
from collections import Counter
from nltk.stem import PorterStemmer
import math
from decimal import *
import numpy as np
import pandas as pd
from IPython.display import display




def truncate_float(float_number, decimal_places):
    multiplier = 10 ** decimal_places
    return int(float_number * multiplier) / multiplier




#CALCULATES THE IG OF EACH WORD
def IG(w, total_pos, total_neg, path_pos, path_neg):
    total_reviews= total_pos + total_neg
    
    
     
    files_positive = Path(path_pos).glob('*.txt')
    files_negative = Path(path_neg).glob('*.txt')
     
    
    
    P1= (total_pos/total_reviews)  #probability of positive reviews
    P0= (total_neg/total_reviews)  #probability of negative reviews
    
    entropy= -P1*math.log2(P1)-P0*math.log2(P0)
    
    
    #Probabilities of C=pos and w=1 & C=neg and w=1
    pos_counter=0
    neg_counter=0 
    total_counter=0
    

    
    for f in files_positive:
        sp= f.read_text().split()
        if w.lower() in [x.lower() for x in sp]:
            pos_counter+=1
            total_counter+=1
      
    for f2 in files_negative:
        sp2= f2.read_text().split()
        if w.lower() in [x.lower() for x in sp2]:
            neg_counter+=1
            total_counter+=1
    
    
    #Probability of w=1 and w=0 
    P_w1 = total_counter/total_reviews
    #print("Probability of ", w, "=1: ", P_w1)
    P_w0 = (total_reviews- total_counter) / total_reviews
    #print("Probability of ", w, "=0: ", P_w0)

        
   
    #Probability of a C=pos and C=neg when w=1
    if P_w1!=0:
        P1_w1= pos_counter/total_counter 
        #print("probability positive reviews with ", w,"=1: ", P1_w1)
        P0_w1= neg_counter/total_counter
        #print("probability negative reviews with ", w,"=1: ", P0_w1)
    else:
        P1_w1=0
        P0_w1=0
    
    
    
    #Probability of a C=pos and C=neg when w=0
    if P_w0!=0:
        P1_w0 = (total_pos-pos_counter)/(total_reviews- total_counter)
        #print("probability positive reviews with ", w,"=0: ", P1_w0)
        P0_w0 = (total_neg-neg_counter)/(total_reviews - total_counter)
        #print("probability negative reviews with ", w,"=0: ", P0_w0)
    else:
        P1_w0 = 0
        P0_w0 = 0
    
    
    entropy_w1= (0 if P1_w1==0 else -P1_w1*math.log2(P1_w1)) - (0 if P0_w1==0 else P0_w1*math.log2(P0_w1))
    #print("Entropy of ", w, "=1: ", entropy_w1)
    entropy_w0= (0 if P1_w0==0 else -P1_w0*math.log2(P1_w0)) - (0 if P0_w0==0 else P0_w0*math.log2(P0_w0))
    #print("Entropy of ", w, "=0: ", entropy_w0)

    
    if entropy!=1.0:
        ig= truncate_float(entropy, 6) - truncate_float((P_w1*entropy_w1 + P_w0*entropy_w0), 6)
    else:
        getcontext().prec = 6
        ig= Decimal(1) - Decimal((P_w1*entropy_w1 + P_w0*entropy_w0))
       
    
    
    #print("IG of ",w , "is: ", ig)
    
    return float(ig)







#FILTERS AN ARRAY OF WORDS BASED ON THEIR IG
def IG_filter(words_filtered, total_pos, total_neg, path_pos, path_neg):
    result=[]
    
    for w in words_filtered:
        ig=IG(w, total_pos, total_neg, path_pos, path_neg)
        if ig >=0.008:
            result.append(w)
    
    return result



In [ ]:
def split_it(text):
    splt = text.split()
    bad_chars = [';', ':'," ", "!", "*", "(", ")", '\"', ".", ",", "/", ">", "<"]

    for i in range(len(splt)):
        for b in bad_chars:
            if b in splt[i]:
                splt[i] = splt[i].replace(b, '')
        splt[i]= splt[i].lower()
    
    return splt

In [ ]:
import os

class tokenizer():

    #FINDING THE MOST USEFUL ATTRIBUTES (WORDS) IN ALL THE TRAINING DATA
    #CONVERTING THE TRAINING DATA INTO VECTORS OF THE ATTRIBUTES WE CHOSE
    def tokenize(self,path_pos, path_neg):
        
        files_positive = Path(path_pos).glob('*.txt')
        files_negative = Path(path_neg).glob('*.txt')
        
        

        total_pos=0
        total_neg=0

        text= ""
        i=0
        for file in files_positive:
            #print(file.name)
            
            text= text+file.read_text()  # the file is opened and closed
            total_pos += 1
            
            
        for file2 in files_negative:
            #print(file.name)
            
            total_neg+=1
            text= text+file2.read_text()  
            


        total_reviews= total_pos+total_neg  

        print("Total positive reviews: ",  total_pos)
        print("Total negative reviews: ", total_neg)



        #FILTERING THE REVIEWS TO KEEP THE USEFUL WORDS
    


        
        splt= split_it(text)
        Counters_found = Counter(splt)
        most_occur = Counters_found.most_common(700)



        ps = PorterStemmer()
        words=[]
        for i in range(len(most_occur)):
            words.append(most_occur[i][0])
                

        bad_chars = [';', ':'," ", "!", "*", "(", ")", '\"', ".", ",", "/", ">", "<"]
        words_to_exclude = ['-', '--', ';', ':', "!", "*", " ", "(", ")", '\"', ".", 'it\'s', 'br', 'mr', 'there\'s', 'your', 'wasn\'t', 'Ms', 'were', 'how', 'get', 'will', 'also', 'been', 'some', 'into', 'because', 'about', 'out', 'me', 'up', 'down', 'my', 'mine', 'their', 'she', 'he\'s', 'the','you', 'an', 'his', 'him', 'her', 'or', 'was', 'have', 'has', 'had', 'in', 'i', 'he', 'we', 'they', 'their', 'theirs', 'which','what', 'where', 'be', 'they', 'has', 'so',  'by', 'who', 'that','this', 'those', 'your', 'these', 'on', 'there', 'and', 'to', 'a', 'it', 'its', 'for', 'if', 'then', 'is', 'at', 'are', 'of', 'no', 'as', 'but', 'with', 'there']    

        words_filtered=[]
        for i, w in enumerate(words):
            
            
            for b in bad_chars:
                if b in words[i]:
                    words[i]= words[i].replace(b,'')
                    w=words[i]
                

            
            
            if w.lower() in words_to_exclude:
                #print('removing ' +w)
                #words.remove(w)
                continue

            words_filtered.append(words[i])        
                


        words_filtered= words_filtered[:len(words_filtered)-128]
        
        
                
        # c=1
        # for i in  words_filtered:
        #     print(c,'. '+i)
        #     c=c+1
            


        
        self.words_final= IG_filter(words_filtered, total_pos, total_neg, path_pos, path_neg)

            
            
        files_positive2 =Path(path_pos).glob('*.txt')
        files_negative2= Path(path_neg).glob('*.txt')
            
        #TOKENIZES EACH TEXT IN THE MOST USEFUL WORDS WE FOUND
        self.vectors= np.zeros([total_reviews, len(self.words_final)+1], dtype=int)
        i=0
        for file in files_positive2:
            i+=1
            for j,w in enumerate(self.words_final):
                if w.lower() in file.read_text().lower():
                    self.vectors[i][j]=1
            self.vectors[i][len(self.words_final)]=1  # initializing the last cell as 1 (category cell)
    
        i=0
        for file2 in files_negative2:
            i+=1
            for j, w in enumerate(self.words_final):
                if w.lower() in file2.read_text().lower():
                    self.vectors[i][j]=1
            #the last cell in the negative reviews is already 0 
        
        
        vectors_df= pd.DataFrame(self.vectors, columns=self.words_final+ ['Category'])
        display(vectors_df)
        
        return self.vectors
    
    
    
    def get_words_final(self):
        return self.words_final
    
    
    
    
    
    
    def tokenize_new(self, path_pos, path_neg ):
        
        files_positive3 =Path(path_pos).glob('*.txt')
        files_negative3= Path(path_neg).glob('*.txt')
        
        pos_count = len(os.listdir(path_pos))
        neg_count = len(os.listdir(path_neg))
        total_count =pos_count + neg_count
        
        new_vectors= np.zeros([total_count, len(self.words_final)+1], dtype=int)
        
        
        i=0
        for f in files_positive3:
            i+=1
            for j,w in enumerate(self.words_final):
                if w.lower() in f.read_text().lower():
                    new_vectors[i][j]=1
            new_vectors[i][len(self.words_final)]=1  # initializing the last cell as 1 (category cell)
    
        i=0
        for f2 in files_negative3:
            i+=1
            for j, w in enumerate(self.words_final):
                if w.lower() in f2.read_text().lower():
                    new_vectors[i][j]=1
            #the last cell in the negative reviews is already 0 
        
        
        
            
        vectors_df= pd.DataFrame(new_vectors, columns=self.words_final+ ['Category'])
        display(vectors_df)
        
        return new_vectors, vectors_df
            
            

In [ ]:

class Probability:
    
    def __init__(self, data, total_pos, total_neg):
        self.data= data
        self.total_pos = total_pos
        self.total_neg = total_neg
    
    
    
    
    
    def prob_tables_training_data(self):
        
        #print("Starting to build the probability tables")
        self.pX_1= np.zeros([2,self.data.shape[1]], dtype=float)
        self.pX_1[1][self.data.shape[1]-1] = 1 
        self.pX_0 = np.zeros([2,self.data.shape[1]], dtype=float)
        self.pX_0[1][self.data.shape[1]-1] = 1 
        
        
        #P(X|C=0)
        for j in range(self.data.shape[1]-1):  #we use -1 in order not to count the probability of the category row too
            X1_C0 = 0
            X0_C0 = 0
            for i in range(self.data.shape[0]):
                if self.data[i][self.data.shape[1]-1] != 0:
                    continue
                
                if self.data[i][j] ==1:
                    X1_C0+=1
                else: 
                    X0_C0+=1
                
                    
            self.pX_1[0][j]= (X1_C0+1)/(self.total_neg+2)
            self.pX_0[0][j] = (X0_C0+1)/(self.total_neg+2)
            #print(self.pX_1[0][j], self.pX_0[0][j])
            
            
        #P(X|C=1)
        for j in range(self.data.shape[1]-1):  #we use -1 in order not to count the probability of the category row too
            X1_C1 = 0
            X0_C1 = 0
            for i in range(self.data.shape[0]):
                if self.data[i][self.data.shape[1]-1] != 1:
                    continue
                
                if self.data[i][j] ==1:
                    X1_C1+=1
                else: 
                    X0_C1+=1
                    
            self.pX_1[1][j]= (X1_C1+1)/(self.total_pos+2)
            self.pX_0[1][j] = (X0_C1+1)/(self.total_pos+2)
            #print(self.pX_1[1][j], self.pX_0[1][j])
    
        
        
        #print("Finished building the probability tables")
    
    
    
    
    # def prob(self, pos,x,c):
    
    #     X_counter=0
    #     C_counter=0
    #     for i in range(self.vectors.shape[0]):
    #         if self.vectors[i][self.vectors.shape[1]-1]!=c:
    #             continue
    #         C_counter+=1
    #         if self.vectors[i][pos]==x:
    #             X_counter+=1 
                
    #     return (X_counter+1)/(C_counter+2)   #using Laplace estimator
     
    
    
    
    def P1_X(self, vector):
        p1= self.total_pos/(self.total_pos+self.total_neg)
        
        pX_C1 =1
        for i, w in enumerate(vector):
            if w==0:
                pX_C1 = pX_C1*self.pX_0[1][i]
            else: 
                pX_C1 = pX_C1*self.pX_1[1][i]
                
            
            
        return pX_C1*p1 



    def P0_X(self, vector):
        p0= self.total_neg/(self.total_pos+self.total_neg)
        
        pX_C0 =1
        for i, w in enumerate(vector):
            if w==0:
                pX_C0 = pX_C0*self.pX_0[0][i]
            else: 
                pX_C0 = pX_C0*self.pX_1[0][i]
            
            
        return pX_C0*p0
    
    
    
    
    
    
    
       
    
    # def P1_X(self, vector, total_pos, total_neg):
    #     p1= total_pos/(total_pos+total_neg)
        
    #     pX_C1= 1
        
    #     for i, w in enumerate(vector):
    #         #word= words_final[i]
    #         if w==0:
    #             pX_C1*= self.prob(i, 0, 1)
    #             print("CALCULATING prob of X when C=1")
    #         else:
    #             pX_C1*=self.prob(i, 1, 1)
    #             print("CALCULATING prob of x=0 when c=0")

    #         if i==(len(vector)-1):
    #             break
    #     return pX_C1
    
    
    
    # def P0_X(self, vector, total_pos, total_neg):
    #     p0= total_neg/(total_pos+total_neg)
        
    #     pX_C0= 1
        
    #     for i, w in enumerate(vector):
    #         #word= words_final[i]
    #         if w==0:
    #             pX_C0*= self.prob(i, 0, 0)
    #             print("CALCULATING prob of x=0 when c=0")
    #         else:
    #             pX_C0*=self.prob(i, 1, 0)
    #             print("CALCULATING prob of x=0 when c=0")

    #         if i==(len(vector)-1):
    #             break
    #     return pX_C0

In [ ]:

def classification_report(y_real, y_pred, p=1):
    #print(y_pred)
    reports= np.zeros([3,5])
    
    true_pos = 0
    false_pos = 0
    true_neg = 0
    false_neg = 0
    support1= 0
    support0= 0
    
    # corr_ident= 0
    # incorr_ident= 0  
    # total_neg= 0
    for i, y in enumerate(y_pred):
        if y== y_real[i] and y==1:
            true_pos+=1
        if y!= y_real[i] and y==1:
            false_pos+=1
        if y!= y_real[i] and y==0:
            false_neg+=1
        if y==y_real[i] and y==0:
            true_neg+=1
        if y_real[i]==1:
            support1+=1
            
    if (true_pos+false_neg) != 0:
        recall= true_pos/(true_pos+false_neg)
    else:
        recall=0
    
    if(true_pos+false_pos) !=0:
        precision = true_pos/ (true_pos +false_pos)
    else:
        precision=0
    
    
    

    # print(recall)
    # print(precision)
    if (precision+recall) !=0 :
        f1_score= 2*(precision*recall)/(precision+recall)
    else:
        f1_score=0
        
    accuracy = (true_pos + true_neg) / len(y_real)
    
    reports[1][0]= recall
    reports[1][1]= precision
    reports[1][2]= f1_score
    
    reports[1][4] = support1
    
    if(true_neg+false_pos)!=0:
        #print('Percentage of negatives found correctly: ', true_neg/(true_neg+false_pos) )
        
        reports[0][3]= true_neg/(true_neg+false_pos)
    
    if (true_pos+false_neg) !=0:
        #print('Percentage of positives found correctly: ', true_pos/(true_pos+false_neg))
        reports[1][3]= true_pos/(true_pos+false_neg)
    
    
    
    # reports[2][0] = accuracy 
    # reports[2][1] = accuracy 
    # reports[2][2] = accuracy 
    # reports[2][3] = len(y_real) 
    
    
    true_pos = 0
    false_pos = 0
    true_neg = 0
    false_neg = 0
    for i, y in enumerate(y_pred):
        if y== y_real[i] and y==0:
            true_pos+=1
        if y!= y_real[i] and y==0:
            false_pos+=1
        if y!= y_real[i] and y==1:
            false_neg+=1
        if y==y_real[i] and y==1:
            true_neg+=1
        if y_real[i]==0:
            support0+=1
    if (true_pos+false_neg) != 0:
        recall= true_pos/(true_pos+false_neg)
    else:
        recall=0
    
    if(true_pos+false_pos) !=0:
        precision = true_pos/ (true_pos +false_pos)
    else:
        precision=0
    
    # print(recall)
    # print(precision)
    if (precision+recall) !=0 :
        f1_score= 2*(precision*recall)/(precision+recall)
    else:
        f1_score=0
    
    reports[0][0]= recall
    reports[0][1]= precision
    reports[0][2]= f1_score
    
    reports[0][4] = support0
    
    
    reports[2][0] = accuracy
    reports[2][1]= accuracy
    reports[2][2] = accuracy
    reports[2][3] = accuracy
    reports[2][4] = support0 + support1
    
    
    
    
    
    
    
    # support_proportion0= support0/len(y_real)    
    # support_proportion1= support1/len(y_real)
    
    
    # macro_avg_p= (reports[0][1]+reports[1][1])/2
    # macro_avg_r= (reports[0][0] + reports[1][0])/2
    # macro_avg_f1 = (reports[0][2] + reports[1][2])/2
    
    # weighted_avg_r = (reports[0][0]*support_proportion1 + reports[1][0]*support_proportion0)
    # weighted_avg_p= (reports[0][1]*support_proportion1 + reports[1][1]*support_proportion0)
    # weighted_avg_f1= (reports[0][2]*support_proportion1 + reports[1][2]*support_proportion0)
    
    # reports[3][0] = macro_avg_r
    # reports[3][1] = macro_avg_p
    # reports[3][2] = macro_avg_f1
    # reports[3][3] = len(y_real)
    
    # reports[4][0]= weighted_avg_r
    # reports[4][1]= weighted_avg_p
    # reports[4][2] = weighted_avg_f1
    # reports[4][3] =len(y_real)
    
    
    reports_df= pd.DataFrame(reports, columns=['Recall', 'Precision', 'F1-score', 'Accuracy', 'Support'], index=['0','1', 'general accuracy'])
    
    if p==1:
        
        print('positives that didn\'t find: ', false_neg)
        print('negatives that didn\'t find: ', false_pos)
        
        display(reports_df)
    
    return reports
    
    

In [ ]:
#SPLIT FUNCTION IN ORDER TO SPLIT DATA INTO X AND Y
def split(vectors):
    x = vectors[:,:-1]
    y = vectors[:,-1]
    return x, y

In [ ]:
class naive_bayes:

        
    
    def fit(self, x, y):
        self.total_pos= 0
        self.total_neg=0
        self.data= np.zeros([x.shape[0],x.shape[1]+1], dtype=int)
        for i in range(x.shape[0]):
            
            for j in range(x.shape[1]):
                #if j== self.data.shape[1]:
                 #   self.data[i][j]= y[i]
                
                self.data[i][j]= x[i][j]
                
            

        for i in range(self.data.shape[0]):
            self.data[i][self.data.shape[1]-1]= y[i]
            if y[i]== 1:
                self.total_pos+=1
            else:
                self.total_neg+=1
                    
        
        #data_df= pd.DataFrame(self.data, columns=words_final+ ['Category'])
        #display(data_df)
        self.p= Probability(self.data, self.total_pos, self.total_neg)
        self.p.prob_tables_training_data()
        return self.data
        
        
        
        
    def predict(self, x_data):
        y_data= [0]*x_data.shape[0]
        #p=Probability(self.data)
        #self.p.prob_tables_training_data()
        #print("starting predictions")
        for i in range(x_data.shape[0]):
            if self.p.P1_X(x_data[i]) > self.p.P0_X(x_data[i]):
                y_data[i]= 1
            else:
                y_data[i]=0
            
        #print("finished predictions")
        return y_data
            
            

In [ ]:
IG("bad",12500, 12500, '/Users/michail/Downloads/aclImdb/train/pos', '/Users/michail/Downloads/aclImdb/train/neg')

In [ ]:

#TRAINING THE ALGORITHM

#files_positive =Path('/Users/michail/Downloads/aclImdb/train/pos').glob('*.txt')
#files_negative= Path('/Users/michail/Downloads/aclImdb/train/neg').glob('*.txt')

path_pos= '/Users/michail/Downloads/aclImdb/train/pos'
path_neg= '/Users/michail/Downloads/aclImdb/train/neg'

t = tokenizer()

train_vectors= t.tokenize(path_pos, path_neg)


In [ ]:
path_pos= '/Users/michail/Downloads/aclImdb/train/pos'
path_neg= '/Users/michail/Downloads/aclImdb/train/neg'

test_vectors, df_study= t.tokenize_new(path_pos, path_neg)

In [ ]:
path_pos= '/Users/michail/Downloads/aclImdb/test/pos'
path_neg= '/Users/michail/Downloads/aclImdb/test/neg'

test_vectors, df_study= t.tokenize_new(path_pos, path_neg)

In [ ]:
df_study.iloc[1]

In [ ]:
x_train, y_train= split(train_vectors)

In [ ]:
nb= naive_bayes()
nb.fit(x_train, y_train)
print('fit done')
y_pred = nb.predict(x_train)


In [ ]:
classification_report(y_train, y_pred)

In [ ]:
x_test, y_test= split(test_vectors)

In [ ]:
y_test_pred = nb.predict(x_test)

In [ ]:
classification_report(y_test, y_test_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_test_pred)

In [ ]:
x_test.shape[0]

In [ ]:
import matplotlib.pyplot as plt
#def plot_learning_curve(x_train, y_test):
    
    
plt.figure()
plt.title("Training curve")
plt.xlabel("Examples")
plt.ylabel("Accuracy")
    
tr_samples= np.linspace(1, x_train.shape[0],100)
    
eval=[]
for sample in tr_samples:
    rep_train = classification_report(y_train[:int(sample)], nb.predict(x_train[:int(sample)]), 0)

    results={
        'sample number': sample,
        'recall': rep_train[0][0],
        'precision': rep_train[0][1],
        'f1-score': rep_train[0][2],
        'general accuracy': rep_train[2][0]
    }
    eval.append(results)
        
        
        
train_recall= [s['recall'] for s in eval]
train_precision= [s['precision'] for s in eval]
train_accuracy = [s['general accuracy'] for s in eval]

train_sample_numbers= [s['sample number'] for s in eval]

plt.plot(train_sample_numbers,train_precision , 'r', label= "Training score")    
    
print("\nTraining sample size: ", train_sample_numbers )
    
test_samples= np.linspace(1, x_test.shape[0], 100)
eval2=[]
for sample in test_samples:
    rep_test= classification_report(y_test[:int(sample)], nb.predict(x_test[:int(sample)]), 0)
    
    results2={
        'sample number': sample,
        'recall': rep_test[0][0],
        'precision': rep_test[0][1],
        'f1-score': rep_test[0][2],
        'general accuracy': rep_test[2][0]
    }
    
    eval2.append(results2)


test_recall= [s['recall'] for s in eval2]
test_precision= [s['precision'] for s in eval2]
test_sample_numbers= [s['sample number'] for s in eval2]
test_accuracy = [s['general accuracy'] for s in eval2]
    
plt.plot(test_sample_numbers,test_precision , 'g', label= "Test score")    
    

    
plt.show()


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

nb = GaussianNB()
nb.fit(x_train, y_train)
print(classification_report(y_train, nb.predict(x_train),
                            zero_division=1))


In [ ]:
print(classification_report(y_test, nb.predict(x_test),
                            zero_division=1))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

nb = RandomForestClassifier()
nb.fit(x_train, y_train)
print(classification_report(y_train, nb.predict(x_train),
                            zero_division=1))


In [ ]:
print(classification_report(y_test, nb.predict(x_test),
                            zero_division=1))